In [1]:
# загрузим библиотеки и установим опции

from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline

from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib
import seaborn as sns


import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

import re
# fix random seed for reproducibility
np.random.seed(7)

Using matplotlib backend: Qt5Agg


In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('data/train_sessions.csv')#,index_col='session_id')
test_df = pd.read_csv('data/test_sessions.csv')#, index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
21668,21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54842,54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77291,77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114020,114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146669,146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [3]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int').astype('str')
test_df[sites] = test_df[sites].fillna(0).astype('int').astype('str')

train_df['list'] = train_df['site1']
test_df['list'] = test_df['site1']
for s in sites[1:]:
    train_df['list'] = train_df['list']+","+train_df[s]
    test_df['list'] = test_df['list']+","+test_df[s]
    
train_df['list_w'] = train_df['list'].apply(lambda x: x.split(','))
test_df['list_w'] = test_df['list'].apply(lambda x: x.split(','))

In [7]:
#Подключаем модуль 
import os 
import json
#Каталог из которого будем брать файлы 
directory = '/home/dz/kaggle/detectiononWeb/data/other_user_logs' 
#Получаем список файлов в переменную files 
files = os.listdir(directory) 

In [8]:
# загрузим словарик сайтов
with open(r"data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'всего сайтов:', sites_dict.shape[0])
sites_dict.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [9]:
site_dict['facebook.com']

2154

In [10]:
sites_dict.ix[3000]

site    vk.com
Name: 3000, dtype: object

In [11]:
%%time
list_ = []
for i in range(len(files)):
    with open('data/other_user_logs/'+files[i]) as data_file:    
        data = pd.read_csv(data_file)
        list_.append(list(data['site'].apply(lambda x: site_dict[x]).apply(str)))
data = pd.read_csv('data/Alice_log.csv')
list_.append(list(data['site'].apply(lambda x: site_dict[x]).apply(str)))

CPU times: user 6.42 s, sys: 140 ms, total: 6.56 s
Wall time: 7.2 s


In [12]:
data = pd.DataFrame()
data['site'] = np.zeros(len(list_))
data['site'] = list_ 

In [ ]:
data.head()

In [ ]:
data['site'][0][0]

In [13]:
from gensim.models import word2vec

In [14]:
model = word2vec.Word2Vec(data['site'], size=400, window=30, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [ ]:
model.wv.most_similar(positive=['3000'])

In [ ]:
sites_dict.ix[7033],sites_dict.ix[7022],sites_dict.ix[31578],sites_dict.ix[12619],sites_dict.ix[31579],sites_dict.ix[7013]

In [ ]:
sites_dict.ix[2997],sites_dict.ix[7015],sites_dict.ix[19132],sites_dict.ix[7033],sites_dict.ix[31441],sites_dict.ix[31564]

In [ ]:
model = word2vec.Word2Vec(train_df['list_w'], size=300, window=3, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [ ]:
model.wv.most_similar(positive=['3000'])

In [ ]:
sites_dict.ix[5302],sites_dict.ix[31578],sites_dict.ix[8666],sites_dict.ix[26385],sites_dict.ix[7033],sites_dict.ix[27228]

In [15]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))
    
    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
   

In [16]:
train_df_mean = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [17]:
y = np.array(train_df['target'])

In [ ]:
def split(train,y,ratio):
    idx = round(train.shape[0] * ratio)
    return train[:idx, :], train[idx:, :], y[:idx], y[idx:]
Xtr, Xval, ytr, yval = split(train_df_mean, y,0.8)
Xtr.shape,Xval.shape,ytr.mean(),yval.mean()

In [ ]:
train_df.shape,test_df.shape

In [ ]:
253561-82797

In [18]:
def get_auc_lr_valid(X, y, C=1, seed=7):
    # разделим выборку на обучающую и валидационную
    idx = 170764
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=4).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [ ]:
train_df_mean_ = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

def CV(train_df_mean,y,estimator):
    r = []
    tscv = TimeSeriesSplit(n_splits=3)
    TimeSeriesSplit(n_splits=3)
    for train, test in tscv.split(train_df_mean):
        Xtr = train_df_mean[train]
        ytr = y[train]
        Xval = train_df_mean[test]
        yval = y[test]
        print(Xtr.shape,Xval.shape,ytr.shape,yval.shape)
        est = estimator.fit(Xtr,ytr)
        z = roc_auc_score(yval, est.predict_proba(Xval)[:, 1])
        print(z)
        r.append(z)
    print(np.mean(r),np.std(r))

In [ ]:
train_df_ = np.concatenate((train_df_mean_, train_df_mean), axis=1)

In [ ]:
CV(train_df_,y,LogisticRegression(C=1, random_state=7, n_jobs=-1))

In [ ]:
920
914
904

In [ ]:
get_auc_lr_valid(train_df_,y)

In [ ]:
get_auc_lr_valid(train_df_mean,y)

In [ ]:
get_auc_lr_valid(train_df_mean_,y)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

class tfidf_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [13]:
data_mean = tfidf_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [ ]:
get_auc_lr_valid(data_mean,y)

In [ ]:
data_mean = tfidf_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])
get_auc_lr_valid(data_mean,y)

In [ ]:
len(site_dict)

In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

max_features = 48371
maxlen = 10
batch_size = 32




In [ ]:
train_df[sites][0:10]

In [ ]:
def get_val(train,y):
    idx = 170764
    Xtr = train[:idx]
    Xval = train[idx:]
    ytr = y[:idx]
    yval = y[idx:]
    return Xtr,Xval,ytr,yval

In [ ]:
Xtr.as_matrix()

In [ ]:
Xtr,Xval,ytr,yval = get_val(train_df[sites],y)

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(Xtr.as_matrix(), ytr,
          batch_size=batch_size,
          epochs=1,
          validation_data=(Xval.as_matrix(), yval),verbose=2)
score, acc = model.evaluate(Xval.as_matrix(), yval,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(Xtr.as_matrix(), ytr,
          batch_size=batch_size,
          epochs=1,
          validation_data=(Xval.as_matrix(), yval),class_weight='auto',verbose=2)
score, acc = model.evaluate(Xval.as_matrix(), yval,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode="binary")


In [ ]:
model.fit(
    X_train, y_train, 
    batch_size=batch_size, 
    nb_epoch=5,
    show_accuracy=True
)

result = model.predict_proba(X)

In [ ]:
model = word2vec.Word2Vec(data['site'], size=300, window=30, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [ ]:
from keras.layers import Embedding, LSTM
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
model3.add(Embedding(20000,128))
model3.add(LSTM(128,dropout = 0.2,recurent_dropout=0.2))
model3.add(Dense(1,activation = 'sigmoid'))

In [ ]:
train_df[sites]

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(10))


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

# new features

In [ ]:
c = train_df['time3'][0]
def isnan(x):
    if x==c:
        return 1
    else:
        return 0
is_nan = train_df['time1'].apply(isnan)
for i in range(2,11):
    is_nan = is_nan + train_df['time'+str(i)].apply(isnan )

In [ ]:
# создадим отдельный датафрейм, где будем работать со временем
time_df = pd.DataFrame(train_df[times]).reset_index()
test_df = pd.DataFrame(test_df[times]).reset_index()
#time_df['index'] = np.array(train_df['session_id'])
time_df['target'] = train_df['target']
# найдем время начала и окончания сессии
#time_df['min'] = train_df[times].min(axis=1)
#time_df['max'] = train_df[times].max(axis=1)
test_df['target'] = -1
time_df = pd.concat([time_df,test_df],axis=0)
# вычислим длительность сессии и переведем в секунды
#time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')

time_df.head()

In [ ]:
dates = pd.to_datetime(time_df['time1'])
time_df['Hour'] = dates.apply(lambda x: x.hour)
time_df['day'] = dates.apply(lambda x: x.day)
time_df['month'] = dates.apply(lambda x: x.month)
time_df['year'] = dates.apply(lambda x: x.year)
time_df['sec_in_week'+'time1'] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_day'+'time1'] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_hour'+"time1"] = dates.apply(lambda x: 60*x.minute+x.second)
time_df.drop(['time1'],axis=1,inplace = True)
for t in times[1:]:
    dates = pd.to_datetime(time_df[t])
    time_df['sec_in_week'+t] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_day'+t] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_hour'+t] = dates.apply(lambda x: 60*x.minute+x.second)
    time_df.drop([t],axis=1,inplace = True)
time_df.head()

In [ ]:
for i in range(len(times)-1):
    t1 = times[i]
    t2 = times[i+1]
    time_df['delta_sec'+t2] = (time_df['sec_in_hour'+t2]-time_df['sec_in_hour'+t1])%3600
time_df.head()

In [ ]:
times_delta = ['delta_sectime%s'% i for i in range(2,11)]
times_delta

In [ ]:
from scipy.stats import kurtosis,skew, mode

In [ ]:
time_df['max-min'] = time_df[times_delta].max(axis=1) - time_df[times_delta].min(axis=1)
time_df['std'] = time_df[times_delta].std(axis=1)
time_df['mean'] = time_df[times_delta].mean(axis=1)
time_df['median'] = time_df[times_delta].median(axis=1)
time_df['max'] = time_df[times_delta].max(axis=1)
time_df['min'] = time_df[times_delta].min(axis=1)
time_df['sem'] = time_df[times_delta].sem(axis=1)
time_df['sum'] = time_df[times_delta].sum(axis=1)
time_df['kurtosis'] = time_df[times_delta].kurtosis(axis=1)
time_df['skew'] = time_df[times_delta].skew(axis=1)
#time_df['mode'] = time_df[times_delta].mode(axis=1)#error
time_df['max+min'] = (time_df['max'] + time_df['min'])*0.5

In [ ]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return -1

In [ ]:
time_df['mean-med'] = time_df['mean'] - time_df['median']
time_df['razmah_75_25'] = time_df['perc_75'] - time_df['perc_25']
time_df['koef_oscil'] = time_df.apply(lambda row: divide(row['max-min'], row['mean']), axis=1)
time_df['koef_oscil_med'] = time_df.apply(lambda row: divide(row['max-min'], row['median']), axis=1)
time_df['koef_var_'] = time_df.apply(lambda row: divide(row['std'], row['mean']), axis=1)
time_df['koef_var_med'] = time_df.apply(lambda row: divide(row['std'], row['median']), axis=1)
time_df['koef_quant_'] = time_df.apply(lambda row: divide(row['razmah_75_25'], row['mean']), axis=1)
time_df['koef_quant_med'] = time_df.apply(lambda row: divide(row['razmah_75_25'], row['median']), axis=1)
time_df['mean/med'] = time_df.apply(lambda row: divide(row['mean'], row['median']), axis=1)

In [ ]:
train_df = np.concatenate((time_df, train_df_mean), axis=1)
test_df = np.concatenate((time_df_t, test_df_mean), axis=1)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True,ngram_range=(1, 5),max_features = 50000)
vectorizer.fit(train_df['list'])

X_train = vectorizer.transform(train_df['list'])
Xtest = vectorizer.transform(test_df['list'])
print(X_train.shape,Xtest.shape)

(253561, 50000) (82797, 50000)


In [19]:
get_auc_lr_valid(X_train,y)

0.89603896283978868

In [ ]:
def get_auc_lr_valid(X, y, C=1, seed=7):
    # разделим выборку на обучающую и валидационную
    idx = 170764
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=4).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [20]:
get_auc_lr_valid(X_train,y)

0.89554681954754056

In [32]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [33]:
full_new_feat = pd.DataFrame(index=full_df.index)

# добавим признак start_month
#full_new_feat['start'] = full_df['time1'].apply(lambda ts: 1000000 * ts.year +ts.month*10000+ ts.day*100+ts.hour)
full_new_feat['start_day_y'] = full_df['time1'].apply(lambda ts: 10000 * ts.year +ts.month*100+ ts.day)
#full_new_feat['start_month'] = full_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
#full_new_feat['start_moth_mh'] = full_df['time1'].apply(lambda ts: ts.month*10000+ ts.day*100+ts.hour)
full_new_feat['start_moth_dh'] = full_df['time1'].apply(lambda ts: ts.day*100+ts.hour)
#full_new_feat['day'] = full_df['time1'].apply(lambda ts: ts.day)
#full_new_feat['minut'] = full_df['time1'].apply(lambda x: x.minute)
full_new_feat['weekday'] = full_df['time1'].apply(lambda x: x.dayofweek)
#full_new_feat['dayofyear'] = full_df['time1'].apply(lambda x: x.dayofyear)
#full_new_feat['weekand'] = full_new_feat['weekday'].apply(weekand)
full_new_feat['start_week'] = 100*full_new_feat['weekday']+full_df['time1'].apply(lambda ts:  ts.hour)
full_new_feat['long'] = (full_df['time2'].apply(lambda ts:  ts.minute)-full_df['time1'].apply(lambda ts:  ts.minute))%3600
full_new_feat['start_hour'] = full_df['time1'].apply(lambda ts: ts.hour)
full_new_feat['morning'] = full_new_feat['start_hour'].apply(lambda hour: int(hour <= 10))
full_new_feat['sec_in_day'+"time1"] = full_df['time1'].apply(lambda x: 60*x.minute+x.second)
for t in times[1:]:
    full_new_feat['sec_in_day'+t] = full_df[t].apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)

In [34]:
for i in range(len(times)-1):
    t1 = times[i]
    t2 = times[i+1]
    full_new_feat['delta_sec'+t2] = (full_new_feat['sec_in_day'+t2]-full_new_feat['sec_in_day'+t1])%3600
full_new_feat.head()

,start_day_y,start_moth_dh,weekday,start_week,long,start_hour,morning,sec_in_daytime1,sec_in_daytime2,sec_in_daytime3,...,sec_in_daytime10,delta_sectime2,delta_sectime3,delta_sectime4,delta_sectime5,delta_sectime6,delta_sectime7,delta_sectime8,delta_sectime9,delta_sectime10
21668,20130112,1208,5,508,0.0,8,1,357,29157.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54842,20130112,1208,5,508,0.0,8,1,2243,31043.0,32827.0,...,NaN,0.0,1784.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
77291,20130112,1208,5,508,0.0,8,1,3013,31814.0,31815.0,...,31817.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
114020,20130112,1208,5,508,0.0,8,1,3017,31817.0,31818.0,...,31820.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
146669,20130112,1208,5,508,0.0,8,1,3020,31820.0,31820.0,...,31822.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
times_delta = ['delta_sectime%s'% i for i in range(2,11)]

In [36]:
from scipy.stats import kurtosis,skew, mode

In [37]:
full_new_feat['max-min'] = full_new_feat[times_delta].max(axis=1) - full_new_feat[times_delta].min(axis=1)
full_new_feat['std'] = full_new_feat[times_delta].std(axis=1)
full_new_feat['mean'] = full_new_feat[times_delta].mean(axis=1)
full_new_feat['median'] = full_new_feat[times_delta].median(axis=1)
full_new_feat['max'] = full_new_feat[times_delta].max(axis=1)
full_new_feat['min'] = full_new_feat[times_delta].min(axis=1)
full_new_feat['sem'] = full_new_feat[times_delta].sem(axis=1)
full_new_feat['sum'] = full_new_feat[times_delta].sum(axis=1)
full_new_feat['kurtosis'] = full_new_feat[times_delta].kurtosis(axis=1)
full_new_feat['skew'] = full_new_feat[times_delta].skew(axis=1)
#time_df['mode'] = time_df[times_delta].mode(axis=1)#error
full_new_feat['max+min'] = (full_new_feat['max'] + full_new_feat['min'])*0.5
full_new_feat.head()

,start_day_y,start_moth_dh,weekday,start_week,long,start_hour,morning,sec_in_daytime1,sec_in_daytime2,sec_in_daytime3,...,std,mean,median,max,min,sem,sum,kurtosis,skew,max+min
21668,20130112,1208,5,508,0.0,8,1,357,29157.0,NaN,...,NaN,0.000000,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0
54842,20130112,1208,5,508,0.0,8,1,2243,31043.0,32827.0,...,1029.416016,595.333333,2.0,1784.0,0.0,594.333614,1786.0,NaN,1.732043,892.0
77291,20130112,1208,5,508,0.0,8,1,3013,31814.0,31815.0,...,0.527046,0.444444,0.0,1.0,0.0,0.175682,4.0,-2.571429,0.271052,0.5
114020,20130112,1208,5,508,0.0,8,1,3017,31817.0,31818.0,...,0.500000,0.333333,0.0,1.0,0.0,0.166667,3.0,-1.714286,0.857143,0.5
146669,20130112,1208,5,508,0.0,8,1,3020,31820.0,31820.0,...,0.440959,0.222222,0.0,1.0,0.0,0.146986,2.0,0.734694,1.619848,0.5


In [39]:
full_new_feat = full_new_feat.fillna(0)

In [40]:

tmp_scaled = StandardScaler().fit_transform(full_new_feat[['start_moth_dh', 'start_hour','start_day_y',\
                                                          'start_week','long','max-min','sec_in_daytime10','delta_sectime10',\
                                                          'std','kurtosis']])


In [41]:
tmp_scaled = StandardScaler().fit_transform(full_new_feat[['start_moth_dh', 'start_hour','start_day_y',\
                                                          'start_week','long','max-min','sec_in_daytime10','delta_sectime10',\
                                                          'std','kurtosis']])


In [1]:
X_tr = csr_matrix(hstack([StandardScaler().fit_transform(X_train),StandardScaler().fit_transform(data_mean)]))
print(get_auc_lr_valid(X_tr, y))

NameError: name 'csr_matrix' is not defined

In [22]:
train_df_mean = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])
test_df_mean = mean_vectorizer(w2v).fit(test_df['list_w']).transform(test_df['list_w'])

In [42]:
X_train.shape,train_df_mean.shape,tmp_scaled.shape

((253561, 50000), (253561, 400), (336358, 10))

In [43]:
tmp_scaled = StandardScaler().fit_transform(full_new_feat[['start_moth_dh', 'start_hour','start_day_y',\
                                                          'start_week','long','max-min','sec_in_daytime10','delta_sectime10',\
                                                          'std','kurtosis']])
X_tr = csr_matrix(hstack([train_df_mean,X_train,tmp_scaled[:idx_split,:]]))


In [44]:
X_te = csr_matrix(hstack([test_df_mean,Xtest,tmp_scaled[idx_split:,:]]))

In [24]:
X_tr = csr_matrix(hstack([X_train,train_df_mean]))
X_te = csr_matrix(hstack([Xtest,test_df_mean]))

In [45]:
X_tr.shape,X_te.shape

((253561, 50410), (82797, 50410))

In [18]:
get_auc_lr_valid(X_tr, y)

0.88938028885705211

In [22]:
get_auc_lr_valid(X_tr, y)

0.89687165764807963

In [46]:
lr = LogisticRegression(C=1, random_state=7, n_jobs=4).fit(X_tr, y)
y_pred = lr.predict_proba(X_te)[:, 1]

In [47]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [48]:
write_to_submission_file(y_pred, 'pred/w2vandtf_idf_time.csv')

In [29]:
# создадим отдельный датафрейм, где будем работать со временем
time_df = pd.DataFrame(train_df[times]).reset_index()
test_df = pd.DataFrame(test_df[times]).reset_index()
#time_df['index'] = np.array(train_df['session_id'])
time_df['target'] = train_df['target']
# найдем время начала и окончания сессии
#time_df['min'] = train_df[times].min(axis=1)
#time_df['max'] = train_df[times].max(axis=1)

# вычислим длительность сессии и переведем в секунды
#time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')

time_df.head()

,index,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,target
0,21668,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0
1,54842,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT,0
2,77291,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17,0
3,114020,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20,0
4,146669,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22,0


In [30]:
test_df['target'] = -1

In [31]:
time_df = pd.concat([time_df,test_df],axis=0)
time_df.shape

(336358, 12)

In [ ]:
tmp_scaled = StandardScaler().fit_transform(full_new_feat[['start_moth_dh', 'start_hour','start_day_y',\
                                            'start_week','long','max-min','sec_in_daytime10','delta_sectime10',\
                                                          'std','kurtosis']])

In [ ]:
dates = pd.to_datetime(time_df['time1'])
time_df['Hour'] = dates.apply(lambda x: x.hour)
time_df['day'] = dates.apply(lambda x: x.day)
time_df['month'] = dates.apply(lambda x: x.month)
time_df['year'] = dates.apply(lambda x: x.year)
time_df['sec_in_week'+'time1'] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_day'+'time1'] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_hour'+"time1"] = dates.apply(lambda x: 60*x.minute+x.second)
time_df.drop(['time1'],axis=1,inplace = True)
for t in times[1:]:
    dates = pd.to_datetime(time_df[t])
    time_df['sec_in_week'+t] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_day'+t] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_hour'+t] = dates.apply(lambda x: 60*x.minute+x.second)
    time_df.drop([t],axis=1,inplace = True)
time_df.head()

In [ ]:
for i in range(len(times)-1):
    t1 = times[i]
    t2 = times[i+1]
    time_df['delta_sec'+t2] = (time_df['sec_in_hour'+t2]-time_df['sec_in_hour'+t1])%3600
time_df.head()

In [ ]:
times_delta = ['delta_sectime%s'% i for i in range(2,11)]
times_delta

In [ ]:
from scipy.stats import kurtosis,skew, mode

In [ ]:
time_df['max-min'] = time_df[times_delta].max(axis=1) - time_df[times_delta].min(axis=1)
time_df['std'] = time_df[times_delta].std(axis=1)
time_df['mean'] = time_df[times_delta].mean(axis=1)
time_df['median'] = time_df[times_delta].median(axis=1)
time_df['max'] = time_df[times_delta].max(axis=1)
time_df['min'] = time_df[times_delta].min(axis=1)
time_df['sem'] = time_df[times_delta].sem(axis=1)
time_df['sum'] = time_df[times_delta].sum(axis=1)
time_df['kurtosis'] = time_df[times_delta].kurtosis(axis=1)
time_df['skew'] = time_df[times_delta].skew(axis=1)
#time_df['mode'] = time_df[times_delta].mode(axis=1)#error
time_df['max+min'] = (time_df['max'] + time_df['min'])*0.5
time_df.head()